<a href="https://colab.research.google.com/github/adarshkumaryadav421-prog/test-repo/blob/main/PCA_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Eigenvalues and Eigenvectors

Welcome to the last assignment of this course and congratulations for making it this far. In this final assignment you will use your knowledge of linear algebra and your skills using Python and NumPy to address some real-world scenarios where linear algebra is actually used to solve and simplify problems.

**After this assignment you will be able to:**
- apply linear transformations, eigenvalues and eigenvectors in a webpage navigation model
- apply PCA on a dataset to reduce its dimensions

## Important Note

Please **do not** delete any exercise cells or add your solutions in different cells. **Keep your solution in the original cell provided**; failing to do so will disrupt the autograder.

Additionally, **do not import any new libraries**, and **avoid importing libraries within any cell designated for grading**, as this will also interfere with the autograder's functionality.

# Table of Contents
- [ 1 - Application of Eigenvalues and Eigenvectors: Navigating Webpages](#1)
  - [ Exercise 1](#ex01)
  - [ Exercise 2](#ex02)
- [ 2 - Application of Eigenvalues and Eigenvectors: Principal Component Analysis](#2)
  - [2.1 Load the data](#2.1)
  - [2.2 Get the covariance matrix](#2.2)
    - [ Exercise 3](#ex03)
    - [ Exercise 4](#ex04)
  - [ 2.3 - Compute the eigenvalues and eigenvectors](#2.3)
  - [ 2.4 Transform the centered data with PCA](#2.4)
    - [ Exercise 5](#ex05)
  - [ 2.5 Analyzing the dimensionality reduction in 2 dimensions](#2.5)
  - [ 2.6 Reconstructing the images from the eigenvectors](#2.6)
  - [ 2.7 Explained variance](#2.7)

## Packages

Run the following cell to load the packages you'll need.

Load the utils module and the unit tests defined for this notebook.

In [ ]:
# Basic utilities installation for Google Colab

# Update pip first
!pip install --upgrade pip

# Data science and machine learning utilities
!pip install pandas numpy matplotlib seaborn plotly
!pip install scikit-learn scipy statsmodels
!pip install jupyter-widgets ipywidgets

# Deep learning frameworks
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install tensorflow keras

# Computer vision utilities
!pip install opencv-python pillow imageio

# Natural language processing
!pip install nltk spacy transformers

# Web scraping and APIs
!pip install requests beautifulsoup4 selenium

# File handling utilities
!pip install openpyxl xlrd h5py

# Visualization enhancements
!pip install wordcloud bokeh altair

# Development utilities
!pip install tqdm joblib pickle5

# Google-specific utilities
!pip install google-colab-utils
!pip install pydrive

# Enable widgets for Jupyter
from google.colab import output
output.enable_custom_widget_manager()

# Import commonly used libraries to verify installation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ All utilities installed successfully!")
print("📊 Ready for data science and machine learning tasks!")

# Optional: Download spacy language model
!python -m spacy download en_core_web_sm

ERROR: Could not find a version that satisfies the requirement jupyter-widgets (from versions: none)
ERROR: No matching distribution found for jupyter-widgets
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached pickle5-0.0.11.tar.gz (132 kB)
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'pickle5' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pickle5'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  E

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse.linalg

Load the utils module and the unit tests defined for this notebook.

<a name='1'></a>
## 1 - Application of Eigenvalues and Eigenvectors: Navigating Webpages

As you learned in the lectures, eigenvalues and eigenvectors play a very important role in what's called (discrete) dynamical systems. As you might recall, a **discrete dynamical system** describes a system where, as time goes by, the state changes according to some process. When defining this dynamical systems you could represent all the possible states, such as sunny, rainy or cloudy, in a vector called the **state vector**.

Each discrete dynamical system can be represented by a transition matrix $P$, which indicates, given a particular state, what are the chances or probabilities of moving to each of the other states. This means the element $(2,1)$ of the matrix represents the probability of transitioning from state $1$ to state $2$.

Starting with an initial state $X_0$, the transition to the next state $X_1$ is a linear transformation defined by the transition matrix $P$: $X_1=PX_0$. That leads to $X_2=PX_1=P^2X_0$, $X_3=P^3X_0$, and so on. This implies that $X_t=PX_{t-1}$ for $t=0,1,2,3,\ldots$. In other words, we can keep multiplying by `P` to move from one state to the next.

One application of discrete dynamical systems is to model browsing web pages. Web pages often contain links to other pages, so the dynamical system would model how a user goes from one page to another by hopping from link to link. For simplicity, assume that the browser is only following links to a new page rather than navigating to an unlinked one.

In this case, the state vector $X_t$ will be the probabilities that the browser is on a particular page at time $t$. Navigation from one page to another advances the model from one state vector $X_{t-1}$ to another state vector $X_t$. A linear transformation, defined by a matrix $P$, will have entries $p_{ij}$ with the probabilities that the browser navigates to page $j$ from page $i$. For fixed column $j$, the entries represent a probability distribution describing location of the browser at the next step, given that you are at state $j$. Thus, the entries in each column must add to 1.

<a name='ex01'></a>
### Exercise 1

For the sake of the example, consider there are only a small number of pages $n=5$. This means that the transition matrix $P$ will be a $5 \times 5$ matrix. In this particular case, all elements on the main diagonal should be equal to $0$, since we are making the reasonable assumption that there is no existing link to the current page. Also, as metioned before, all the entries in each column must add to one. Here is an example of such a matrix for $n=5$:

$$P=
\begin{bmatrix}
0    & 0.75 & 0.35 & 0.25 & 0.85 \\
0.15 & 0    & 0.35 & 0.25 & 0.05 \\
0.15 & 0.15 & 0    & 0.25 & 0.05 \\
0.15 & 0.05 & 0.05 & 0    & 0.05 \\
0.55 & 0.05 & 0.25 & 0.25 & 0
\end{bmatrix}\tag{5}
$$

Define vector $X_0$, so the browser starts navigation at page $4$ ($X_0$ is a vector with a single entry equal to one, and all other entries equal to zero). Apply the transformation once: $X_1=PX_0$ to find a vector of the probabilities that the browser is at each of four pages.

In [ ]:
import numpy as np

P = np.array([
    [0, 0.75, 0.35, 0.25, 0.85],
    [0.15, 0, 0.35, 0.25, 0.05],
    [0.15, 0.15, 0, 0.25, 0.05],
    [0.15, 0.05, 0.05, 0, 0.05],
    [0.55, 0.05, 0.25, 0.25, 0]
])

X0 = np.array([[0],[0],[0],[1],[0]])

### START CODE HERE ###

# Multiply matrix P and X_0 (matrix multiplication).
X1 = P @ X0
# Alternative: X1 = np.dot(P, X0)

### END CODE HERE ###

print(f'Sum of columns of P: {sum(P)}')
print(f'X1:\n{X1}')

Sum of columns of P: [1. 1. 1. 1. 1.]
X1:
[[0.25]
 [0.25]
 [0.25]
 [0.  ]
 [0.25]]


In [ ]:
import numpy as np

def test_matrix(P, X0, X1):
    """
    Manual test function to replace w4_unittest.test_matrix(P, X0, X1)
    Tests matrix multiplication and validates results
    """

    print("🧪 Running manual tests for matrix operations...")
    print("=" * 50)

    # Test 1: Check if X1 is the result of P @ X0
    expected_X1 = P @ X0

    print("Test 1: Matrix Multiplication Correctness")
    print(f"Expected X1:\n{expected_X1}")
    print(f"Your X1:\n{X1}")

    if np.allclose(X1, expected_X1):
        print("✅ PASS: X1 = P @ X0 is correct")
    else:
        print("❌ FAIL: X1 is not equal to P @ X0")
        print(f"Difference:\n{X1 - expected_X1}")

    print("-" * 30)

    # Test 2: Check matrix dimensions
    print("Test 2: Matrix Dimensions")
    print(f"P shape: {P.shape}")
    print(f"X0 shape: {X0.shape}")
    print(f"X1 shape: {X1.shape}")

    if P.shape == (5, 5) and X0.shape == (5, 1) and X1.shape == (5, 1):
        print("✅ PASS: All matrices have correct dimensions")
    else:
        print("❌ FAIL: Matrix dimensions are incorrect")

    print("-" * 30)

    # Test 3: Check if P is a valid transition matrix (columns sum to 1)
    print("Test 3: Transition Matrix Properties")
    col_sums = np.sum(P, axis=0)
    print(f"Column sums of P: {col_sums}")

    if np.allclose(col_sums, 1.0):
        print("✅ PASS: P is a valid transition matrix (columns sum to 1)")
    else:
        print("❌ FAIL: P is not a valid transition matrix")

    print("-" * 30)

    # Test 4: Check if X0 is a valid state vector (sums to 1)
    print("Test 4: Initial State Vector Properties")
    x0_sum = np.sum(X0)
    print(f"Sum of X0: {x0_sum}")

    if np.isclose(x0_sum, 1.0):
        print("✅ PASS: X0 is a valid probability vector (sums to 1)")
    else:
        print("❌ FAIL: X0 does not sum to 1")

    print("-" * 30)

    # Test 5: Check if X1 is a valid state vector (sums to 1)
    print("Test 5: Result State Vector Properties")
    x1_sum = np.sum(X1)
    print(f"Sum of X1: {x1_sum}")

    if np.isclose(x1_sum, 1.0):
        print("✅ PASS: X1 is a valid probability vector (sums to 1)")
    else:
        print("❌ FAIL: X1 does not sum to 1")

    print("-" * 30)

    # Test 6: Manual calculation verification
    print("Test 6: Manual Calculation Verification")
    # Since X0 = [0,0,0,1,0], X1 should be the 4th column of P
    expected_column = P[:, 3].reshape(-1, 1)
    print(f"4th column of P (expected result):\n{expected_column}")

    if np.allclose(X1, expected_column):
        print("✅ PASS: X1 matches the 4th column of P (correct for X0=[0,0,0,1,0])")
    else:
        print("❌ FAIL: X1 does not match expected column")

    print("=" * 50)

    # Overall result
    all_tests = [
        np.allclose(X1, expected_X1),
        P.shape == (5, 5) and X0.shape == (5, 1) and X1.shape == (5, 1),
        np.allclose(col_sums, 1.0),
        np.isclose(x0_sum, 1.0),
        np.isclose(x1_sum, 1.0),
        np.allclose(X1, expected_column)
    ]

    if all(all_tests):
        print("🎉 \033[92m All tests passed \033[0m")
    else:
        failed_tests = sum(1 for test in all_tests if not test)
        print(f"⚠️  {failed_tests} out of {len(all_tests)} tests failed")

    return all(all_tests)

# Example usage:
# Define your matrices (replace with your actual values)
P = np.array([
    [0, 0.75, 0.35, 0.25, 0.85],
    [0.15, 0, 0.35, 0.25, 0.05],
    [0.15, 0.15, 0, 0.25, 0.05],
    [0.15, 0.05, 0.05, 0, 0.05],
    [0.55, 0.05, 0.25, 0.25, 0]
])

X0 = np.array([[0],[0],[0],[1],[0]])

# Your solution (replace None with your answer)
X1 = P @ X0

# Run the test
test_matrix(P, X0, X1)

🧪 Running manual tests for matrix operations...
Test 1: Matrix Multiplication Correctness
Expected X1:
[[0.25]
 [0.25]
 [0.25]
 [0.  ]
 [0.25]]
Your X1:
[[0.25]
 [0.25]
 [0.25]
 [0.  ]
 [0.25]]
✅ PASS: X1 = P @ X0 is correct
------------------------------
Test 2: Matrix Dimensions
P shape: (5, 5)
X0 shape: (5, 1)
X1 shape: (5, 1)
✅ PASS: All matrices have correct dimensions
------------------------------
Test 3: Transition Matrix Properties
Column sums of P: [1. 1. 1. 1. 1.]
✅ PASS: P is a valid transition matrix (columns sum to 1)
------------------------------
Test 4: Initial State Vector Properties
Sum of X0: 1
✅ PASS: X0 is a valid probability vector (sums to 1)
------------------------------
Test 5: Result State Vector Properties
Sum of X1: 1.0
✅ PASS: X1 is a valid probability vector (sums to 1)
------------------------------
Test 6: Manual Calculation Verification
4th column of P (expected result):
[[0.25]
 [0.25]
 [0.25]
 [0.  ]
 [0.25]]
✅ PASS: X1 matches the 4th column of P (

True

Applying the transformation $m$ times you can find a vector $X_m$ with the probabilities of the browser being at each of the pages after $m$ steps of navigation.

In [ ]:
X = np.array([[0],[0],[0],[1],[0]])
m = 20

for t in range(m):
    X = P @ X

print(X)

[[0.39392366]
 [0.13392366]
 [0.11407667]
 [0.0850993 ]
 [0.27297672]]


It is useful to predict the probabilities in $X_m$ when $m$ is large, and thus determine what pages a browser is more likely to visit after a long period of browsing the web. In other words, we want to know which pages ultimately get the most traffic. One way to do that is just apply the transformation many times, and with this small $5 \times 5$ example you can do that just fine. In real life problems, however, you'll have enormous matrices and doing so will be computationally expensive. Here is where eigenvalues and eigenvectors can help here significantly reducing the amount of calculations. Let's see how!

Begin by finding eigenvalues and eigenvectors for the previously defined matrix $P$:

In [ ]:
eigenvals, eigenvecs = np.linalg.eig(P)
print(f'Eigenvalues of P:\n{eigenvals}\n\nEigenvectors of P\n{eigenvecs}')

Eigenvalues of P:
[ 1.         -0.70367062  0.00539505 -0.08267227 -0.21905217]

Eigenvectors of P
[[-0.76088562 -0.81362074  0.10935376  0.14270615 -0.39408574]
 [-0.25879453  0.050269   -0.6653158   0.67528802 -0.66465044]
 [-0.2204546   0.07869601 -0.29090665  0.17007443  0.35048734]
 [-0.1644783   0.12446953  0.19740707 -0.43678067  0.23311487]
 [-0.52766004  0.56018621  0.64946163 -0.55128793  0.47513398]]


As you can see, there is one eigenvalue with value $1$, and the other four have an aboslute values smaller than 1. It turns out this is a property of transition matrices. In fact, they have so many properties that these types of matrices fall into a category of matrices called **Markov matrix**.

In general, a square matrix whose entries are all nonnegative, and the sum of the elements for each column is equal to $1$ is called a **Markov matrix**. Markov matrices have a handy property - they always have an eigenvalue equal to 1. As you learned in the lectures, in the case of transition matrices, the eigenvector associated with the eigenvalue $1$ will determine the state of the model in the long run , after evolving for a long period of time.

You can easily verify that the matrix $P$ you defined earlier is in fact a Markov matrix.
So, if $m$ is large enough, the equation $X_m=PX_{m-1}$ can be rewritten as $X_m=PX_{m-1}=1\times X_m$. This means that predicting probabilities at time $m$, when $m$ is large you can simply just look for an eigenvector corresponding to the eigenvalue $1$.

So, let's extract the eigenvector associated to the eigenvalue $1$.

In [ ]:
X_inf = eigenvecs[:,0]

print(f"Eigenvector corresponding to the eigenvalue 1:\n{X_inf[:,np.newaxis]}")

Eigenvector corresponding to the eigenvalue 1:
[[-0.76088562]
 [-0.25879453]
 [-0.2204546 ]
 [-0.1644783 ]
 [-0.52766004]]


<a name='ex02'></a>
### Exercise 2

Just to verify the results, perform matrix multiplication $PX$ (multiply matrix `P` and vector `X_inf`) to check that the result will be equal to the vector $X$ (`X_inf`).

In [ ]:
# This is organised as a function only for grading purposes.
def check_eigenvector(P, X_inf):
    ### START CODE HERE ###
    X_check = P @ X_inf
    ### END CODE HERE ###
    return X_check

X_check = check_eigenvector(P, X_inf)
print("Original eigenvector corresponding to the eigenvalue 1:\n" + str(X_inf))
print("Result of multiplication:" + str(X_check))

# Function np.isclose compares two NumPy arrays element by element, allowing for error tolerance (rtol parameter).
print("Check that PX=X element by element:" + str(np.isclose(X_inf, X_check, rtol=1e-10)))

Original eigenvector corresponding to the eigenvalue 1:
[-0.76088562 -0.25879453 -0.2204546  -0.1644783  -0.52766004]
Result of multiplication:[-0.76088562 -0.25879453 -0.2204546  -0.1644783  -0.52766004]
Check that PX=X element by element:[ True  True  True  True  True]


In [ ]:
def manual_test_check_eigenvector():
    """
    Manual test for the check_eigenvector function.
    Uses a simple 2x2 matrix and its known eigenvector for eigenvalue 1.
    """
    print("🧪 Running manual test for check_eigenvector...")

    # Define a simple 2x2 transition matrix
    # Columns sum to 1, and it has an eigenvalue of 1
    P_test = np.array([
        [0.5, 0.3],
        [0.5, 0.7]
    ])

    # Define the eigenvector for eigenvalue 1 for P_test
    # You can calculate this manually or using np.linalg.eig
    # For P_test, the eigenvector for eigenvalue 1 is proportional to [0.3, 0.5]
    # We normalize it so the sum is 1, representing probabilities
    X_inf_test = np.array([[0.375], [0.625]]) # Normalized [0.3, 0.5]

    print(f"Test matrix P_test:\n{P_test}")
    print(f"Test eigenvector X_inf_test (for eigenvalue 1):\n{X_inf_test}")

    # Call the function to be tested
    X_check_test = check_eigenvector(P_test, X_inf_test)

    print(f"Result of P_test @ X_inf_test:\n{X_check_test}")

    # Check if the result is close to the original eigenvector
    is_close = np.allclose(X_inf_test, X_check_test, rtol=1e-10)

    print(f"Check that P_test @ X_inf_test is close to X_inf_test: {is_close}")

    if is_close:
        print("✅ PASS: check_eigenvector test passed")
    else:
        print("❌ FAIL: check_eigenvector test failed")
        print(f"Difference:\n{X_inf_test - X_check_test}")

# Run the manual test
manual_test_check_eigenvector()

🧪 Running manual test for check_eigenvector...
Test matrix P_test:
[[0.5 0.3]
 [0.5 0.7]]
Test eigenvector X_inf_test (for eigenvalue 1):
[[0.375]
 [0.625]]
Result of P_test @ X_inf_test:
[[0.375]
 [0.625]]
Check that P_test @ X_inf_test is close to X_inf_test: True
✅ PASS: check_eigenvector test passed


This result gives the direction of the eigenvector, but as you can see the entries can't be interpreted as probabilities since you have negative values, and they don't add to 1. That's no problem. Remember that by convention `np.eig` returns eigenvectors with norm 1, but actually any vector on the same line is also an eigenvector to the eigenvalue 1, so you can simply scale the vector so that all entries are positive and add to one.This will give you the long-run probabilities of landing on a given web page.

In [ ]:
X_inf = X_inf/sum(X_inf)
print(f"Long-run probabilities of being at each webpage:\n{X_inf[:,np.newaxis]}")

Long-run probabilities of being at each webpage:
[[0.39377747]
 [0.13393269]
 [0.11409081]
 [0.08512166]
 [0.27307736]]


This means that after navigating the web for a long time, the probability that the browser is at page 1 is 0.394, of being on page 2 is 0.134, on page 3 0.114, on page 4 0.085, and finally page 5 has a probability of 0.273.

Looking at this result you can conclude that page 1 is the most likely for the browser to be at, while page 4 is the least probable one.

If you compare the result of `X_inf` with the one you got after evolving the systems 20 times, they are the same up to the third decimal!

Here is a fun fact: this type of a model was the foundation of the PageRank algorithm, which is the basis of Google's very successful search engine.

<a name='2'></a>
## 2 - Application of Eigenvalues and Eigenvectors: Principal Component Analysis

As you learned in the lectures, one of the useful applications of eigenvalues and eigenvectors is the dimensionality reduction algorithm called Principal Component Analyisis, or PCA for short.

In this second section of the assignment you will be applying PCA on an image dataset to perform image compression.

You will be using a portion of the [Cat and dog face](https://www.kaggle.com/datasets/alessiosanna/cat-dog-64x64-pixel/data) dataset from Kaggle. In particular, you will be using the cat images.

Remember that to apply PCA on any dataset you will begin by defining the covariance matrix. After that you will compute the eigenvalues and eigenvectors of this covariance matrix. Each of these eigenvectors will be a **principal component**. To perform the dimensionality reduction, you will take the $k$ principal components associated to the $k$ biggest eigenvalues, and transform the original data by projecting it onto the direction of these principal components (eigenvectors).

<a name='2.1'></a>
### 2.1 - Load the data
Begin by loading the images and transforming them to black and white using `load_images` function from utils.

In [ ]:
# Graded cell
def center_data(Y):
    """
    Center your original data
    Args:
         Y (ndarray): input data. Shape (n_observations x n_pixels)
    Outputs:
        X (ndarray): centered data
    """
    ### START CODE HERE ###
    mean_vector = np.mean(Y, axis=0)
    # use np.reshape to reshape into a matrix with the same size as Y. Remember to use order='F'
    mean_matrix = np.tile(mean_vector, (Y.shape[0], 1))

    X = Y - mean_matrix
    ### END CODE HERE ###
    return X

In [ ]:
# Graded cell
def get_cov_matrix(X):
    """ Calculate covariance matrix from centered data X
    Args:
        X (np.ndarray): centered data matrix
    Outputs:
        cov_matrix (np.ndarray): covariance matrix
    """

    ### START CODE HERE ###
    # Get the number of observations (rows) and features (columns)
    n_observations, n_features = X.shape

    # Calculate the covariance matrix using the formula: (X.T @ X) / (n_observations - 1)
    cov_matrix = (X.T @ X) / (n_observations - 1)
    ### END CODE HERE ###

    return cov_matrix

In [ ]:
# GRADED cell
def perform_PCA(X, eigenvecs, k):
    """
    Perform dimensionality reduction with PCA
    Inputs:
        X (ndarray): original data matrix. Has dimensions (n_observations)x(n_variables)
        eigenvecs (ndarray): matrix of eigenvectors. Each column is one eigenvector. The k-th eigenvector
                            is associated to the k-th eigenvalue
        k (int): number of principal components to use
    Returns:
        Xred
    """

    ### START CODE HERE ###
    # Select the first k eigenvectors (principal components)
    V = eigenvecs[:, :k]

    # Project the centered data onto the selected eigenvectors
    Xred = X @ V
    ### END CODE HERE ###
    return Xred